# Imports

In [1]:
import blocksnet as bn
%matplotlib inline
from shapely.geometry import Point
from geopandas import GeoDataFrame
from blocksnet.models.land_use import LandUse

c:\Users\rkozl\Desktop\BlocksNet\blocksnet\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Model

## Читаем модель

In [2]:
city = bn.City.from_pickle(r'C:\Users\rkozl\Desktop\BlocksNet\blocksnet\small_model.pickle')

Выбираем квартал который хотим менять и выбираем новое землепользование для него

In [3]:
selected_block_id = 104
selected_block = city[selected_block_id]
orig_landuse = selected_block.land_use
new_landuse = LandUse.TRANSPORT

In [4]:
blocks_landuse_dict ={
    104:LandUse.TRANSPORT,
    10:LandUse.RESIDENTIAL
}

# Optimizer

Инициализируем класс для подбора оптимальной застройки сервисов в соответсвии с выбранным землепользование

In [5]:
opt = bn.BlockOptimizer(city_model=city)
result = opt.calculate(blocks_landuse_dict)

ProvisionOpt:
MAXIMIZE
0.000132870353966623*104_0 + 0.0002214505899443716*104_1 + 0.07280219780219781*104_13 + 0.10920329670329672*104_14 + 0.1820054945054945*104_15 + 0.00017305399342279457*104_16 + 0.0002884233223713243*104_17 + 0.00031726565460845675*104_18 + 4.160988073775983e-05*104_19 + 0.00031003082592212023*104_2 + 0.00011095968196735954*104_20 + 2.3101965607641763e-06*104_21 + 6.930589682292528e-06*104_22 + 3.234275185069847e-05*104_23 + 0.00024235373951820077*104_24 + 0.00048470747903640155*104_25 + 0.0012117686975910038*104_26 + 6.908756058732324e-05*104_27 + 9.869651512474747e-05*104_28 + 0.00019739303024949494*104_29 + 0.1107252949721858*104_3 + 0.0013359339514254416*104_33 + 0.002671867902850883*104_34 + 0.013359339514254415*104_35 + 0.045355587808418*104_36 + 0.090711175616836*104_37 + 0.90711175616836*104_38 + 0.07086418878219893*104_4 + 0.163873436558835*104_5 + 0.3011728023243454*104_6 + 2.7053837135900443*104_7 + 4.508972855983407*104_8 + 5.861664712778429*104_9 + -7

c:\Users\rkozl\Desktop\BlocksNet\blocksnet\.venv\Lib\site-packages\pulp\pulp.py:1704: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


ProvisionOpt:
MAXIMIZE
0.000132870353966623*104_0 + 0.0002214505899443716*104_1 + 0.07280219780219781*104_13 + 0.10920329670329672*104_14 + 0.1820054945054945*104_15 + 0.00017305399342279457*104_16 + 0.0002884233223713243*104_17 + 0.00031726565460845675*104_18 + 4.160988073775983e-05*104_19 + 0.00031003082592212023*104_2 + 0.00011095968196735954*104_20 + 2.3101965607641763e-06*104_21 + 6.930589682292528e-06*104_22 + 3.234275185069847e-05*104_23 + 0.00024235373951820077*104_24 + 0.00048470747903640155*104_25 + 0.0012117686975910038*104_26 + 6.908756058732324e-05*104_27 + 9.869651512474747e-05*104_28 + 0.00019739303024949494*104_29 + 0.1107252949721858*104_3 + 0.0013359339514254416*104_33 + 0.002671867902850883*104_34 + 0.013359339514254415*104_35 + 0.045355587808418*104_36 + 0.090711175616836*104_37 + 0.90711175616836*104_38 + 0.07086418878219893*104_4 + 0.163873436558835*104_5 + 0.3011728023243454*104_6 + 2.7053837135900443*104_7 + 4.508972855983407*104_8 + 5.861664712778429*104_9 + -7

Метод выдает оптимальные типы сервисов и их вместимость

In [6]:
result['optimal_update_df']

,dog_park,landfill,bakery,bus_stop
104,424.0,1890000.0,0.0,0.0
10,0.0,0.0,196500.0,28.0


In [7]:
for key in result['bricks_to_build_df']:
    print(f"block index {key}")
    print(result['bricks_to_build_df'][key])
    print()

block index 104
   service_type  capacity      area  is_integrated  service_counts
13     dog_park       424    1600.0          False             4.0
38     landfill   1890000  378000.0          False           189.0

block index 10
    service_type  capacity     area  is_integrated  service_counts
162       bakery    196500  78600.0           True           655.0
178     bus_stop         8      8.0          False             1.0
179     bus_stop        20     20.0          False             2.0



Также можно узнать какие сервисы были снесены перед постройкой новых

In [8]:
result['deleting_df']

,cemetery,embankment,park,fuel,religion,hospital,museum,university,market,population,pitch,restaurant,convenience,bank,lawyer,cafe,multifunctional_center,hotel,parking
0,-207.0,-700.0,-290,-740.0,-1172.0,-6062.0,-6620.0,-7429.0,-734.0,-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,-278,0.0,0.0,0.0,0.0,0.0,0.0,-1175,-28.0,-540.0,-511.0,-63.0,-250.0,-45.0,-2705.0,-655.0,-18.0


И посмотреть число обеспеченности

In [9]:
result['provision']

748.1486520455451

С помощью метода plot мы можем отобразить гистограмму изменений в в квартале по сравнению с оригинальной застройкой

In [10]:
opt.plot(**result)

TypeError: BlockOptimizer.plot() got an unexpected keyword argument 'deleting_df'

Также можно выбрать оптимальное землепользование использовав метод для всех возможных типов землепользования

In [ ]:
best_landuse = selected_block.land_use
temp_diff = 0
for lanuse in LandUse:
    if lanuse == orig_landuse:
        continue
    result = opt.calculate(selected_block=selected_block, new_landuse=lanuse)
    diff = result['total_after'] - result['total_before']
    print(f'{selected_block.land_use} -> {lanuse}: {round(diff, 3)}')
    if diff > temp_diff:
        temp_diff = diff
        best_landuse = lanuse

print(f'best lanuse {best_landuse}')